#### The purpose of this EDA notebook is the following:
- Better understand the nature of the relationship between the independent variables
- Create an initial model with reasonable economic assumptions that may be dropped in later versions
- Explore methods of imputation for missing variables to provide more data samples
- Avoid linear combinations that might be more difficult to spot in the Bayesian Modeling process
- Establish a reasonable measure of variable importance, which along with correlation plots may inform initial hierarchies
- Create visualizations of poor quality data and also establish probability distributions for the likelihood function

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('darkgrid')
from rfpimp import *
from rfpimp import plot_corr_heatmap
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from scipy.optimize import curve_fit
from scipy.stats import linregress
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.inspection import permutation_importance
import category_encoders as ce

#### In an effort to generalize for future EDA, 'df' makes it easier to debug

In [2]:
df = pd.read_csv('C:/Users/norri/Desktop/all_ab_data.csv')
df.describe()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4393 entries, 0 to 4392
Columns: 123 entries, % Dollar Sales by Merch Any Display to Weighted Average Base Price Per Unit
dtypes: float64(107), int64(6), object(10)
memory usage: 4.1+ MB


### The info function above tells us we have three data types and counts of each. The next section will explore those variables in groups.

In [3]:
df = df.drop(['CONCAT', 'Brewer Value'], axis=1)
segment = [var for var in df.columns if df[var].dtype == 'O']
print('There are {} categorical variables\n'.format(len(segment)))
print('The categorical variables are :\n\n', segment)
print(df[segment].isnull().sum() / len(df))
df_cat = df.select_dtypes(include=object)
df_cat.info()
df_cat.describe()


There are 8 categorical variables

The categorical variables are :

 ['Brand Family Value', 'Brand Value', 'Category Value', 'Geography', 'Product', 'Segment Value', 'Time', 'Type Value']
Brand Family Value    0.208969
Brand Value           0.529706
Category Value        0.016390
Geography             0.000000
Product               0.000000
Segment Value         0.140451
Time                  0.000000
Type Value            0.298657
dtype: float64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4393 entries, 0 to 4392
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Brand Family Value  3475 non-null   object
 1   Brand Value         2066 non-null   object
 2   Category Value      4321 non-null   object
 3   Geography           4393 non-null   object
 4   Product             4393 non-null   object
 5   Segment Value       3776 non-null   object
 6   Time                4393 non-null   object
 7   Type V

,Brand Family Value,Brand Value,Category Value,Geography,Product,Segment Value,Time,Type Value
count,3475,2066,4321,4393,4393,3776,4393,3081
unique,41,31,3,15,72,6,6,8
top,MICHELOB ULTRA FAMILY,BUD LIGHT,BEER,AWG Corp-SRMA - Food,BUD LIGHT,CRAFT/IMPORT,Week Ending 01-02-22,LIGHT
freq,332,90,4248,359,90,969,740,928


In [4]:
df = df.drop(
    ['F81', 'Product Development Index', 'Sum of Dollar Sales Checkout Display',
     'Sum of Dollar Sales Outside Display', 'Sum of Dollar Sales Signage and Feature',
     'Sum of Dollar Sales Signage, Feature and Display'], axis=1)
integer = [var for var in df.columns if df[var].dtype == 'int64']
print('There are {} integer variables\n'.format(len(integer)))
print('The integer variables are :\n\n', integer)
print(df[integer].isnull().sum())
df_int = df.select_dtypes(include=int)
if len(df_int.columns) > 0.0:
    df_int.info()
    df_int.describe()

There are 0 integer variables

The integer variables are :

 []
Series([], dtype: float64)


In [5]:
df = df.drop(
    ['Unit Share of SubCategory', 'Dollar Share of SubCategory',
     'Incremental Dollars', 'Incremental Dollars % Change vs YA',
     'Incremental Units', 'Incremental Units % Change vs YA',
     '% Increase in Dollars by Merch Any Special Pack',
     '% Increase in Dollars by Merch Special Pack Only',
     '% Increase in Units by Merch',
     '% Increase in Units by Merch Any Special Pack',
     '% Increase in Units by Merch Special Pack Only',
     '% Incremental Units by Merch Any Special Pack',
     '% Incremental Units by Merch Special Pack Only',
     '% Increase in Units by Merch No Merch'], axis=1)
fp = [var for var in df.columns if df[var].dtype == 'float64']
print('There are {} float variables\n'.format(len(fp)))
print('The float variables are :\n\n', fp)
fp_na = df[fp].isnull().sum() / len(df) * 100
print(fp_na[fp_na > 10])
fp_zero = df[fp].sum()
print(fp_zero[fp_zero == 0.0])

There are 93 float variables

The float variables are :

 ['% Dollar Sales by Merch Any Display', '% Dollar Sales by Merch Any Feature', '% Dollar Sales by Merch Any Merch', '% Dollar Sales by Merch Any Price Reduction', '% Increase in Dollars by Merch Advertised Frequent Shopper', '% Increase in Dollars by Merch Any Display', '% Increase in Dollars by Merch Any Feature', '% Increase in Dollars by Merch Any Merch', '% Increase in Dollars by Merch Any Price Reduction', '% Increase in Dollars by Merch Display Only', '% Increase in Dollars by Merch Feature and Display', '% Increase in Dollars by Merch Feature and/or Display', '% Increase in Dollars by Merch Feature Only', '% Increase in Dollars by Merch Price Reductions Only', '% Increase in Units by Merch Advertised Frequent Shopper', '% Increase in Units by Merch Any Display', '% Increase in Units by Merch Any Feature', '% Increase in Units by Merch Any Merch', '% Increase in Units by Merch Any Price Reduction', '% Increase in Units by 

In [6]:
# agreed upon drops
df = df.drop(
    ['Dollar Sales', 'Sum of Base Dollar Sales',
     'Sum of Incremental Dollars', 'Base Dollar Sales',
     'Base Dollar Sales % Change vs YA', 'Base Unit Sales'], axis=1)

In [7]:
# can drop year ago columns
df_fp = df.select_dtypes('float')
df_num = df_fp
df_fp.describe()

,% Dollar Sales by Merch Any Display,% Dollar Sales by Merch Any Feature,% Dollar Sales by Merch Any Merch,% Dollar Sales by Merch Any Price Reduction,% Increase in Dollars by Merch Advertised Frequent Shopper,% Increase in Dollars by Merch Any Display,% Increase in Dollars by Merch Any Feature,% Increase in Dollars by Merch Any Merch,% Increase in Dollars by Merch Any Price Reduction,% Increase in Dollars by Merch Display Only,...,Total Points of Distribution,Unit Sales,Unit Sales % Change vs YA,Unit Sales per Pt of Distribution,Unit Share of Category,Unit Share of Category Change vs YA,Units per Store Selling,Units per Store Selling % Change vs YA,Units per Store Selling Year Ago,Weighted Average Base Price Per Unit
count,4393.000000,4393.000000,4393.000000,4393.000000,4393.000000,4393.000000,4393.000000,4393.000000,4393.000000,4393.000000,...,4393.000000,4393.000000,3917.000000,4393.000000,4393.000000,4393.000000,4393.000000,4393.000000,3917.000000,4393.000000
mean,0.068785,0.046816,0.138043,0.060164,0.001455,0.063270,0.025865,0.066125,0.087068,0.055928,...,103.461055,2172.335267,1.528799,39.682057,0.789307,0.090372,8.997648,-0.000511,11.177418,11.538373
std,0.147870,0.162260,0.228133,0.146462,0.031127,0.305275,0.169769,0.360176,0.425199,0.298333,...,183.163335,7636.025807,28.101594,105.168853,2.477898,1.500920,21.363508,0.004789,25.671570,4.474148
min,0.000000,0.000000,0.000000,0.000000,0.000000,-0.880920,-0.778573,-0.951749,-0.951749,-0.880920,...,0.025329,0.178856,-0.996255,0.436618,0.000032,-13.473599,0.171188,-0.008952,0.312438,1.239082
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.008061,0.000000,0.000000,...,7.290226,35.000000,-0.448000,3.168485,0.020356,-0.039655,2.000000,-0.002354,2.535714,9.173739
50%,0.000000,0.000000,0.021026,0.001036,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,37.094045,194.000000,-0.198113,9.208365,0.109133,-0.001882,3.157895,-0.000910,4.000000,11.886448
75%,0.055210,0.000000,0.167083,0.045176,0.000000,0.000000,0.000000,0.090597,0.067896,0.000000,...,112.573300,979.000000,-0.005540,31.266267,0.372046,0.027209,6.914634,0.000089,8.485380,14.682571
max,1.000000,1.000000,1.000000,1.000000,1.041233,3.158592,3.290151,4.666404,8.573740,4.067986,...,2037.747538,109507.510248,1278.886585,1980.829850,34.740099,27.798110,345.449559,0.100000,394.195739,24.990000


### Clearly the float variables contain all the missing data in the dataset. In
### cases like this, dropping the missing values are a trade-off to consider
### against dropping an entire column and losing its input into the model

In [ ]:
# this section should come later; for larger datasets takes too long to run
# with too little return in information
cols = 5
rows = 20
num_cols = df_num.select_dtypes(exclude='object').columns
fig = plt.figure(figsize=(cols * 5, rows * 5))
for i, col in enumerate(num_cols):
    ax = fig.add_subplot(rows, cols, i + 1)
    sns.histplot(x=df[col], ax=ax)
fig.tight_layout()
plt.show()

In [ ]:
# these visualizations provide some clue as to how to model the large number
# of variables and to examine their relationships
corr = df_num.corr(method="pearson")
corr.style.background_gradient(cmap="coolwarm").set_precision(2)

##### Imputation of missing values is too unreliable to base the rest of the
##### model on. Later tests will tell if there is any bias present. This is when
##### the most standout variables should be chosen for the model

In [ ]:
# this is just another visualization of correlation that I like, I may switch it with others
# is nice because it is easier to zoom in for inspection
viz = plot_corr_heatmap(df_num, figsize=(20, 20))
viz.view()

In [ ]:
corr_imp = corr[abs(corr['Sum of Dollar Sales']) >.5]
corr_imp = corr_imp[['Sum of Dollar Sales']]
print(corr_imp)

In [ ]:
figure, axes = plt.subplots(2, 3, figsize=(15, 15))
sns.kdeplot(ax=axes[0, 0], x='Sum of Dollar Sales', data=df_num)
sns.kdeplot(ax=axes[0, 1], x='Dollar Share of Category', data=df_num)
sns.kdeplot(ax=axes[0, 2], x='Sum of Dollar Sales Any Display', data=df_num)
sns.kdeplot(ax=axes[1, 0], x='Sum of Dollar Sales Any Merch', data=df_num)
sns.kdeplot(ax=axes[1, 1], x='Sum of Dollar Sales No Merch (non-promo)', data=df_num)
sns.kdeplot(ax=axes[1, 2], x='Sum of Dollar Sales Any Price Reduction', data=df_num)
plt.show()

### These kde plots will a primary tool in determining the likelihood
### distributions and giving information on the prior

In [ ]:
features = ['Sum of Dollar Sales Any Merch', 'Dollar Share of Category',
            'Sum of Dollar Sales Any Display',
            'Sum of Dollar Sales No Merch (non-promo)',
            'Sum of Dollar Sales Any Price Reduction',
            'Sum of Dollar Sales Feature and/or Display',
            'Dollar Sales per Pt of Distribution',
            'Units per Store Selling',
            'Avg Weekly Units per Store Selling']
target = 'Sum of Dollar Sales'

df_all = df_num.dropna().astype(dtype='int32')
df_all = df_all[features + [target]]
df_train, df_test = train_test_split(df_all, test_size=0.15)

X_train, y_train = df_train.drop('Sum of Dollar Sales', axis=1), df_train['Sum of Dollar Sales']
X_test, y_test = df_test.drop('Sum of Dollar Sales', axis=1), df_test['Sum of Dollar Sales']

rf = RandomForestClassifier(n_estimators=100, n_jobs=-1,
                            max_features=1.0,
                            min_samples_leaf=10, oob_score=True)
rf.fit(X_train, y_train)

In [ ]:
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features=1.0, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_samples_leaf=10, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
                       oob_score=True, random_state=None, verbose=0, warm_start=False)

I = importances(rf, X_test, y_test)
plot_importances(I, width=12, vscale=3)

### This importance plot from a RandomForestClassifier is one of the key
### ways we'll understand which variables are the most important

#### With an array of all the variables, I could make an importance plot
#### for everything but the categorical variables, but I would have to change
#### the dependent variable each time. Remember that we dropped missing values
#### instead of dropping columns, which we could do with a column that did not
#### make an impression on the correlation plot or importance plot

In [ ]:
I = pd.DataFrame()

I['Feature'] = X_train.columns
I['Importance'] = rf.feature_importances_
I = I.sort_values('Importance', ascending=False)
I = I.set_index('Feature')
viz = plot_importances(I, width=16, vscale=4)

#### Our categorical variables shouldn't be forgotten; just alter the threshold
#### down from 250 if the axis titles start to look messy

In [ ]:
for col in df.select_dtypes(include='object'):
    if df[col].nunique() <= 250:
        sns.countplot(y=col, data=df)
        plt.show()